In [1]:
# !pip install -U pip
# !pip install sklearn
# !pip install numpy
# !pip install azure-storage-blob
# !pip install opencv-python
# !pip install torch torchvision
# !pip install timeout_decorator

In [1]:
from model import DeepVONet

In [2]:
from deepvo import *


timeouts set to 5


In [3]:

if USE_TPU:
    VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
    !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
    !python pytorch-xla-env-setup.py --version $VERSION

In [4]:
# torch.set_num_interop_threads(6)
# torch.set_num_threads(32)


In [5]:
torch.cuda.is_available()

False

In [6]:
envlist = get_environment_list()
print('Find {} environments..'.format(len(envlist)))
print(envlist)

Find 18 environments..
['abandonedfactory/', 'abandonedfactory_night/', 'amusement/', 'carwelding/', 'endofworld/', 'gascola/', 'hospital/', 'japanesealley/', 'neighborhood/', 'ocean/', 'office/', 'office2/', 'oldtown/', 'seasidetown/', 'seasonsforest/', 'seasonsforest_winter/', 'soulcity/', 'westerndesert/']


In [7]:
envlist = ['abandonedfactory/', 'abandonedfactory_night/', 'amusement/', 'carwelding/', 'endofworld/', 'gascola/', 'hospital/', 'japanesealley/', 'neighborhood/', 'ocean/', 'office/', 'office2/', 'oldtown/', 'seasidetown/', 'seasonsforest/', 'seasonsforest_winter/', 'soulcity/', 'westerndesert/']

In [8]:
diff_level = 'Hard'
env_ind = 0
trajlist = get_trajectory_list(envlist[env_ind], easy_hard = diff_level)
print('Find {} trajectories in {}'.format(len(trajlist), envlist[env_ind]+diff_level))
print(trajlist)

Find 12 trajectories in abandonedfactory/Hard
['abandonedfactory/Hard/P000/', 'abandonedfactory/Hard/P001/', 'abandonedfactory/Hard/P002/', 'abandonedfactory/Hard/P003/', 'abandonedfactory/Hard/P004/', 'abandonedfactory/Hard/P005/', 'abandonedfactory/Hard/P006/', 'abandonedfactory/Hard/P007/', 'abandonedfactory/Hard/P008/', 'abandonedfactory/Hard/P009/', 'abandonedfactory/Hard/P010/', 'abandonedfactory/Hard/P011/']


In [9]:
global_data_list = []

In [12]:
# [[ global_data_list.extend(get_trajectory_list(env, easy_hard=diff_level)) for diff_level in ["Hard", "Easy"]] for env in envlist]

In [13]:
# train_list, test_list = train_test_split(global_data_list, test_size=10, random_state=12)

In [14]:
# train_list, val_list = train_test_split(train_list, test_size=10, random_state=12)

In [15]:
# from google.colab import drive
# drive.mount('/content/drive')

In [16]:
# kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {} #check again
# val_batch_size = 2#len(val_list)
# val_datapath = [get_image_list(trajectory, left_right="left") for trajectory in val_list[:2]]

# test_loader = torch.utils.data.DataLoader(VisualOdometryDataLoader(val_datapath, 
#                                                                 trajectory_length=10, 
#                                                                 transform=preprocess, 
#                                                                 test=True), 
#                                           batch_size=val_batch_size, 
#                                           shuffle=False, **kwargs)
# for batch_idx, (images_stacked, odometries_stacked) in tqdm(enumerate(test_loader)):
#     images_stacked = images_stacked.permute(1, 0, 2, 3, 4)

#     for t in range(10):
#         f, axarr = plt.subplots(1,2, figsize=(25,25))
#         axarr[0].imshow(unnormalize(images_stacked[t][0][:3]).data.cpu().numpy().transpose((1, 2, 0)))
#         axarr[1].imshow(unnormalize(images_stacked[t][0][3:]).data.cpu().numpy().transpose((1, 2, 0)))
#         plt.show()
    

In [17]:
# # if __name__ == "__main__":
# parser = argparse.ArgumentParser(description='PyTorch on Place Recognition + Visual Odometry')

# parser.add_argument('--mode', default='train', type=str, help='support option: train/test')
# parser.add_argument('--datapath', default='datapath', type=str, help='path KITII odometry dataset')
# parser.add_argument('--bsize', default=32, type=int, help='minibatch size')
# parser.add_argument('--trajectory_length', default=10, type=int, help='trajectory length')
# parser.add_argument('--lr', type=float, default=0.0001, metavar='LR', help='learning rate (default: 0.0001)')
# parser.add_argument('--momentum', type=float, default=0.5, metavar='M', help='SGD momentum (default: 0.5)')
# parser.add_argument('--weight_decay', type=float, default=1e-4, metavar='M', help='SGD momentum (default: 0.5)')
# parser.add_argument('--tau', default=0.001, type=float, help='moving average for target network')
# parser.add_argument('--debug', dest='debug', action='store_true')
# parser.add_argument('--train_iter', default=20000000, type=int, help='train iters each timestep')
# parser.add_argument('--validation_steps', default=100, type=int, help='test iters each timestep')
# parser.add_argument('--epsilon', default=50000, type=int, help='linear decay of exploration policy')
# parser.add_argument('--checkpoint_path', default=None, type=str, help='Checkpoint path')
# parser.add_argument('--checkpoint', default=None, type=str, help='Checkpoint')
# args = parser.parse_args()

In [18]:
chkpnts = ['checkpoint_1_hospital.Hard.P040..carwelding.Hard.P001..soulcity.Hard.P002..abandonedfactory_night.Hard.P011..pth',
 'checkpoint_1_gascola.Easy.P003..endofworld.Easy.P008..hospital.Easy.P017..carwelding.Easy.P006..pth',
 'checkpoint_1_westerndesert.Easy.P010..westerndesert.Easy.P008..seasonsforest_winter.Easy.P005..seasonsforest_winter.Easy.P002..pth',
 'checkpoint_1_hospital.Hard.P042..oldtown.Easy.P002..office2.Hard.P010..westerndesert.Easy.P011..seasonsforest.Hard.P001..gascola.Hard.P001..amusement.Hard.P000..office2.Hard.P001..pth',
 'checkpoint_1_abandonedfactory.Hard.P009..gascola.Hard.P003..gascola.Hard.P008..abandonedfactory_night.Hard.P005..abandonedfactory_night.Easy.P009..westerndesert.Hard.P001..pth',
 'checkpoint_1_abandonedfactory_night.Hard.P003..neighborhood.Hard.P012..hospital.Easy.P000..amusement.Hard.P007..seasidetown.Easy.P005..hospital.Easy.P032..seasonsforest_winter.Easy.P001..office.Hard.P004..pth',
 'checkpoint_1_hospital.Easy.P023..neighborhood.Easy.P001..abandonedfactory_night.Hard.P006..endofworld.Easy.P000..office2.Hard.P009..hospital.Easy.P007..westerndesert.Easy.P007..abandonedfactory_night.Easy.P006..pth',
 'checkpoint_1_gascola.Easy.P007..westerndesert.Easy.P005..seasonsforest_winter.Easy.P007..abandonedfactory_night.Hard.P010..hospital.Easy.P009..hospital.Easy.P012..ocean.Easy.P009..abandonedfactory.Easy.P000..pth',
 'checkpoint_1_seasonsforest_winter.Easy.P008..office.Easy.P002..soulcity.Hard.P001..westerndesert.Hard.P006..pth']

def generate_valid_graph(trajectory_length=10,validation_steps=10):
    kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {} #check again
    val_batch_size = 2#len(val_list)
    val_datapath = [get_image_list(trajectory, left_right="left") for trajectory in val_list[:2]]
    test_loader = torch.utils.data.DataLoader(VisualOdometryDataLoader(val_datapath, 
                                                                    trajectory_length=trajectory_length, 
                                                                    transform=preprocess, 
                                                                    test=True), 
                                              batch_size=val_batch_size, 
                                              shuffle=False, **kwargs)
    losses = []
    if USE_CUDA:
        model.cuda()
    val_loss = test(model, val_list, trajectory_length=trajectory_length, validation_steps=validation_steps, preprocess=preprocess, test_loader=test_loader)
    losses.append(val_loss)    
    for chkpnt in os.listdir("checkpoints_v2/"):
        print("loading: " + chkpnt)
        checkpoint = torch.load("checkpoints_v2/"+chkpnt, map_location=torch.device('cpu'))
        model.load_state_dict(checkpoint['state_dict'])
        if USE_CUDA:
            model.cuda()
        val_loss = test(model, val_list, trajectory_length=trajectory_length, validation_steps=validation_steps, preprocess=preprocess, test_loader=test_loader)
        losses.append(val_loss)
    plt.plot(range(len(losses)), losses)
    plt.show()


In [19]:
 class fake_arg_parser(argparse.ArgumentParser):
    def __init__(self,checkpoint_path, datapath, 
                 mode='train', checkpoint=None,
                 bsize=16, trajectory_length=8, 
                 lr=0.0003, momentum=0.5, 
                 weight_decay=1e-4, tau=0.001, 
                 train_iter=1, validation_steps=100,
                 epsilon=50000):
        self.mode = mode
        self.datapath = datapath
        self.bsize = bsize
        self.checkpoint_path =checkpoint_path
        self.checkpoint = checkpoint
        self.lr = lr
        self.momentum = momentum
        self.weight_decay =weight_decay
        self.tau =tau
        self.train_iter = train_iter
        self.validation_steps = validation_steps
        self.epsilon = epsilon
        self.trajectory_length = trajectory_length

In [20]:
# from google.colab import drive
# drive.mount('/content/drive')

In [21]:
train_list = ['office/Hard/P003/',
 'seasonsforest/Hard/P006/',
 'neighborhood/Hard/P009/',
 'seasonsforest/Easy/P002/',
 'hospital/Easy/P012/',
 'westerndesert/Hard/P007/',
 'carwelding/Easy/P006/',
 'office2/Easy/P010/',
 'abandonedfactory_night/Easy/P012/',
 'ocean/Easy/P010/',
 'westerndesert/Hard/P001/',
 'hospital/Easy/P014/',
 'hospital/Easy/P035/',
 'hospital/Hard/P038/',
 'office2/Easy/P007/',
 'hospital/Hard/P049/',
 'seasidetown/Hard/P001/',
 'abandonedfactory_night/Hard/P009/',
 'ocean/Hard/P003/',
 'seasidetown/Hard/P000/',
 'ocean/Easy/P006/',
 'westerndesert/Hard/P006/',
 'seasonsforest/Easy/P001/',
 'abandonedfactory_night/Hard/P002/',
 'seasonsforest/Hard/P002/',
 'westerndesert/Easy/P001/',
 'seasonsforest/Hard/P004/',
 'hospital/Easy/P010/',
 'seasonsforest_winter/Hard/P016/',
 'westerndesert/Easy/P011/',
 'hospital/Easy/P011/',
 'seasonsforest_winter/Easy/P006/',
 'abandonedfactory_night/Hard/P011/',
 'seasidetown/Easy/P003/',
 'hospital/Easy/P017/',
 'seasidetown/Easy/P002/',
 'hospital/Easy/P004/',
 'abandonedfactory/Easy/P009/',
 'westerndesert/Easy/P012/',
 'hospital/Easy/P013/',
 'hospital/Easy/P033/',
 'hospital/Easy/P025/',
 'hospital/Hard/P045/',
 'abandonedfactory/Hard/P006/',
 'gascola/Hard/P000/',
 'office2/Hard/P001/',
 'carwelding/Hard/P003/',
 'office2/Hard/P006/',
 'carwelding/Easy/P007/',
 'westerndesert/Easy/P005/',
 'soulcity/Hard/P009/',
 'soulcity/Easy/P003/',
 'hospital/Easy/P022/',
 'japanesealley/Hard/P004/',
 'hospital/Easy/P008/',
 'neighborhood/Easy/P015/',
 'seasidetown/Hard/P003/',
 'hospital/Easy/P028/',
 'abandonedfactory_night/Easy/P009/',
 'ocean/Easy/P009/',
 'hospital/Easy/P027/',
 'gascola/Hard/P001/',
 'hospital/Easy/P024/',
 'office2/Easy/P000/',
 'hospital/Easy/P019/',
 'gascola/Easy/P001/',
 'hospital/Easy/P002/',
 'hospital/Easy/P023/',
 'ocean/Hard/P005/',
 'seasonsforest/Easy/P011/',
 'westerndesert/Easy/P002/',
 'japanesealley/Hard/P002/',
 'abandonedfactory_night/Hard/P006/',
 'carwelding/Easy/P004/',
 'japanesealley/Hard/P000/',
 'seasonsforest_winter/Easy/P008/',
 'seasonsforest_winter/Hard/P015/',
 'hospital/Hard/P037/',
 'seasidetown/Easy/P001/',
 'hospital/Easy/P009/',
 'hospital/Easy/P036/',
 'carwelding/Easy/P005/',
 'westerndesert/Easy/P008/',
 'hospital/Hard/P040/',
 'ocean/Easy/P005/',
 'seasonsforest/Easy/P008/',
 'hospital/Easy/P031/',
 'hospital/Easy/P026/',
 'oldtown/Hard/P004/',
 'carwelding/Hard/P000/',
 'abandonedfactory_night/Easy/P002/',
 'westerndesert/Easy/P007/',
 'seasonsforest/Easy/P010/',
 'seasonsforest/Easy/P007/',
 'abandonedfactory_night/Hard/P014/',
 'seasonsforest_winter/Easy/P009/',
 'westerndesert/Hard/P004/',
 'abandonedfactory_night/Hard/P001/',
 'endofworld/Easy/P001/',
 'hospital/Easy/P003/',
 'hospital/Hard/P046/',
 'office2/Hard/P007/',
 'westerndesert/Easy/P010/',
 'seasonsforest_winter/Hard/P014/',
 'ocean/Easy/P001/',
 'soulcity/Hard/P001/',
 'endofworld/Hard/P005/',
 'seasonsforest/Hard/P005/',
 'hospital/Easy/P021/',
 'hospital/Easy/P016/',
 'ocean/Hard/P004/',
 'hospital/Hard/P048/',
 'hospital/Easy/P005/',
 'office/Hard/P006/',
 'neighborhood/Hard/P013/',
 'soulcity/Easy/P007/',
 'endofworld/Hard/P002/',
 'oldtown/Hard/P006/',
 'seasidetown/Easy/P004/',
 'carwelding/Hard/P002/',
 'japanesealley/Easy/P007/',
 'westerndesert/Hard/P003/',
 'amusement/Hard/P000/',
 'ocean/Hard/P009/',
 'ocean/Hard/P001/',
 'neighborhood/Easy/P002/',
 'ocean/Easy/P012/',
 'abandonedfactory_night/Hard/P010/',
 'ocean/Easy/P002/',
 'seasonsforest_winter/Hard/P013/',
 'amusement/Easy/P007/',
 'hospital/Hard/P047/',
 'neighborhood/Hard/P007/',
 'amusement/Hard/P001/',
 'westerndesert/Easy/P006/',
 'office2/Easy/P005/',
 'endofworld/Easy/P007/',
 'endofworld/Easy/P009/',
 'seasonsforest/Hard/P001/',
 'amusement/Hard/P007/',
 'hospital/Hard/P042/',
 'soulcity/Easy/P002/',
 'ocean/Easy/P011/',
 'neighborhood/Hard/P014/',
 'office2/Hard/P010/',
 'abandonedfactory/Hard/P001/',
 'soulcity/Hard/P002/',
 'seasidetown/Easy/P000/',
 'soulcity/Hard/P008/',
 'hospital/Easy/P000/',
 'ocean/Hard/P008/',
 'oldtown/Hard/P005/',
 'hospital/Easy/P034/',
 'hospital/Easy/P015/',
 'soulcity/Hard/P004/',
 'hospital/Hard/P044/',
 'neighborhood/Hard/P006/',
 'japanesealley/Easy/P005/',
 'hospital/Easy/P030/',
 'neighborhood/Hard/P003/',
 'hospital/Hard/P041/',
 'westerndesert/Hard/P005/',
 'endofworld/Easy/P008/',
 'japanesealley/Hard/P005/',
 'ocean/Easy/P013/',
 'endofworld/Easy/P003/',
 'abandonedfactory_night/Hard/P005/',
 'abandonedfactory_night/Easy/P010/',
 'westerndesert/Easy/P009/',
 'office2/Easy/P003/',
 'neighborhood/Hard/P015/',
 'hospital/Easy/P006/',
 'ocean/Hard/P002/',
 'seasidetown/Easy/P005/',
 'endofworld/Hard/P001/',
 'abandonedfactory/Hard/P011/',
 'abandonedfactory_night/Easy/P005/',
 'seasonsforest/Easy/P004/',
 'hospital/Easy/P029/',
 'oldtown/Easy/P004/',
 'amusement/Easy/P008/',
 'soulcity/Hard/P000/',
 'soulcity/Easy/P009/',
 'office/Hard/P001/',
 'japanesealley/Easy/P002/',
 'japanesealley/Hard/P001/',
 'gascola/Hard/P003/',
 'ocean/Hard/P007/',
 'office/Hard/P004/',
 'seasonsforest_winter/Hard/P012/',
 'amusement/Hard/P003/',
 'office2/Hard/P002/',
 'office2/Easy/P011/',
 'gascola/Hard/P007/',
 'seasonsforest/Easy/P009/',
 'neighborhood/Easy/P005/',
 'oldtown/Hard/P007/',
 'soulcity/Easy/P012/',
 'abandonedfactory/Easy/P006/',
 'amusement/Easy/P001/',
 'ocean/Hard/P006/',
 'gascola/Hard/P009/',
 'abandonedfactory/Hard/P003/',
 'japanesealley/Easy/P004/',
 'soulcity/Hard/P003/',
 'endofworld/Easy/P002/',
 'oldtown/Hard/P000/',
 'amusement/Hard/P005/',
 'amusement/Hard/P006/',
 'abandonedfactory_night/Hard/P008/',
 'oldtown/Easy/P005/',
 'office2/Hard/P008/',
 'carwelding/Hard/P001/',
 'westerndesert/Hard/P002/',
 'abandonedfactory_night/Easy/P001/',
 'japanesealley/Easy/P003/',
 'neighborhood/Easy/P017/',
 'seasonsforest_winter/Easy/P001/',
 'abandonedfactory_night/Hard/P003/',
 'neighborhood/Easy/P007/',
 'seasidetown/Easy/P008/',
 'neighborhood/Hard/P017/',
 'oldtown/Hard/P001/',
 'office/Hard/P007/',
 'abandonedfactory/Hard/P004/',
 'seasidetown/Hard/P002/',
 'office/Hard/P005/',
 'neighborhood/Easy/P012/',
 'office2/Easy/P006/',
 'seasidetown/Easy/P009/',
 'endofworld/Hard/P000/',
 'abandonedfactory/Hard/P008/',
 'westerndesert/Hard/P000/',
 'abandonedfactory/Easy/P002/',
 'oldtown/Hard/P002/',
 'seasidetown/Hard/P004/',
 'seasidetown/Easy/P007/',
 'office/Easy/P005/',
 'neighborhood/Easy/P018/',
 'seasonsforest_winter/Easy/P004/',
 'carwelding/Easy/P001/',
 'seasonsforest_winter/Easy/P002/',
 'endofworld/Easy/P004/',
 'neighborhood/Hard/P011/',
 'abandonedfactory_night/Easy/P006/',
 'westerndesert/Easy/P013/',
 'office/Hard/P002/',
 'abandonedfactory/Hard/P007/',
 'hospital/Easy/P007/',
 'ocean/Hard/P000/',
 'hospital/Easy/P032/',
 'abandonedfactory/Hard/P005/',
 'seasidetown/Easy/P006/',
 'neighborhood/Hard/P016/',
 'neighborhood/Easy/P010/',
 'office2/Hard/P009/',
 'abandonedfactory_night/Hard/P007/',
 'amusement/Hard/P004/',
 'seasonsforest_winter/Easy/P003/',
 'japanesealley/Easy/P001/',
 'soulcity/Easy/P005/',
 'neighborhood/Hard/P002/',
 'office/Easy/P006/',
 'soulcity/Easy/P004/',
 'westerndesert/Easy/P004/',
 'abandonedfactory/Easy/P005/',
 'abandonedfactory_night/Easy/P003/',
 'endofworld/Easy/P006/',
 'soulcity/Easy/P001/',
 'neighborhood/Easy/P021/',
 'neighborhood/Easy/P009/',
 'neighborhood/Easy/P004/',
 'abandonedfactory/Hard/P010/',
 'ocean/Easy/P004/',
 'hospital/Hard/P043/',
 'abandonedfactory_night/Easy/P004/',
 'soulcity/Easy/P011/',
 'neighborhood/Hard/P008/',
 'office2/Hard/P005/',
 'seasonsforest_winter/Easy/P007/',
 'abandonedfactory/Hard/P000/',
 'oldtown/Hard/P008/',
 'neighborhood/Hard/P012/',
 'ocean/Easy/P008/',
 'seasonsforest_winter/Hard/P017/',
 'neighborhood/Hard/P004/',
 'endofworld/Easy/P005/',
 'abandonedfactory_night/Hard/P013/',
 'carwelding/Easy/P002/',
 'ocean/Easy/P000/',
 'soulcity/Easy/P000/',
 'soulcity/Easy/P008/',
 'abandonedfactory/Easy/P004/',
 'abandonedfactory_night/Hard/P000/',
 'amusement/Easy/P002/',
 'amusement/Easy/P004/',
 'neighborhood/Easy/P016/',
 'abandonedfactory_night/Hard/P012/',
 'office/Easy/P001/',
 'gascola/Hard/P005/',
 'gascola/Hard/P002/',
 'office/Easy/P000/',
 'seasonsforest_winter/Hard/P011/',
 'neighborhood/Easy/P020/',
 'neighborhood/Easy/P008/',
 'gascola/Hard/P008/',
 'soulcity/Easy/P010/',
 'office2/Easy/P008/',
 'office2/Hard/P003/',
 'gascola/Hard/P006/',
 'gascola/Easy/P003/',
 'endofworld/Easy/P000/',
 'neighborhood/Hard/P010/',
 'amusement/Easy/P006/',
 'abandonedfactory_night/Easy/P007/',
 'amusement/Easy/P003/',
 'seasonsforest_winter/Hard/P010/',
 'office/Easy/P004/',
 'office2/Easy/P009/',
 'office/Easy/P002/',
 'oldtown/Easy/P001/',
 'neighborhood/Easy/P014/',
 'abandonedfactory/Easy/P008/',
 'neighborhood/Easy/P019/',
 'oldtown/Easy/P000/',
 'office2/Easy/P004/',
 'oldtown/Easy/P002/',
 'seasonsforest_winter/Easy/P005/',
 'office/Easy/P003/',
 'abandonedfactory_night/Easy/P008/',
 'gascola/Hard/P004/',
 'oldtown/Easy/P007/',
 'abandonedfactory_night/Easy/P011/',
 'soulcity/Easy/P006/',
 'neighborhood/Hard/P001/',
 'neighborhood/Easy/P003/',
 'abandonedfactory/Hard/P009/',
 'abandonedfactory/Easy/P000/',
 'neighborhood/Hard/P000/',
 'seasonsforest_winter/Easy/P000/',
 'gascola/Easy/P007/',
 'neighborhood/Easy/P013/',
 'gascola/Easy/P006/',
 'abandonedfactory_night/Easy/P013/',
 'neighborhood/Easy/P001/',
 'gascola/Easy/P004/',
 'abandonedfactory/Easy/P011/',
 'abandonedfactory/Easy/P010/',
 'neighborhood/Easy/P000/']

In [22]:
args = fake_arg_parser("checkpoints_v2", train_list)#, checkpoint="checkpoints_v2/checkpoint_1_seasonsforest.Easy.P008..amusement.Easy.P006..pth"  )

In [23]:
if USE_TPU:
    import torch_xla
    import torch_xla.core.xla_model as xm
    device_tpu = xm.xla_device()
model = DeepVONet()

NameError: name 'USE_TPU' is not defined

In [ ]:
# kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {} #check again
# val_batch_size = 2#len(val_list)
# val_datapath = [get_image_list(trajectory, left_right="left") for trajectory in val_list[:2]]

# test_loader = torch.utils.data.DataLoader(VisualOdometryDataLoader(val_datapath, 
#                                                                 trajectory_length=10, 
#                                                                 transform=preprocess, 
#                                                                 test=True), 
#                                           batch_size=val_batch_size, 
#                                           shuffle=False, **kwargs)
# for batch_idx, (images_stacked, odometries_stacked) in tqdm(enumerate(test_loader)):
#     images_stacked = images_stacked.permute(1, 0, 2, 3, 4)

#     for t in range(10):
#         f, axarr = plt.subplots(1,2, figsize=(25,25))
#         axarr[0].imshow(unnormalize(images_stacked[t][0][:3]).data.cpu().numpy().transpose((1, 2, 0)))
#         axarr[1].imshow(unnormalize(images_stacked[t][0][3:]).data.cpu().numpy().transpose((1, 2, 0)))
#         plt.show()
    

In [ ]:
# generate_valid_graph()

In [13]:
if args.checkpoint is not None:
    print("loading")
    checkpoint = torch.load(args.checkpoint, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['state_dict'])
if USE_CUDA:
    model.cuda()
if USE_TPU:
    model = model.to(device_tpu)


loading


FileNotFoundError: [Errno 2] No such file or directory: 'checkpoints_v2/checkpoint_1_seasonsforest.Easy.P008..amusement.Easy.P006..pth'

In [41]:
# args = parser.parse_args()

In [42]:
# from google.colab import drive
# drive.mount('/content/drive')

In [43]:

# train_loader = torch.utils.data.DataLoader(VisualOdometryDataLoader(['hospital/Easy/P011/', 'office2/Hard/P005/', 'abandonedfactory_night/Hard/P014/', 'seasonsforest_winter/Easy/P009/'],
#                                                                     trajectory_length=4, transform=preprocess), batch_size=args.bsize, shuffle=False, drop_last=True)


In [44]:
# test(model, val_list, 10, 10, preprocess)

In [45]:
# ans = predict_trajectory(model, [val_list[0]], 10, 340, preprocess)

In [46]:
# fe = open("pose_estimations/est.txt", "w")
# fr = open("pose_estimations/real.txt", "w")
# for est, real in zip(ans[0], ans[1]):
#     fe.write(" ".join(["{:e}".format(x) for x in est[0]]))
#     fe.write("\n")
#     fr.write(" ".join(["{:e}".format(x) for x in real]))
#     fr.write("\n")
        
# fe.close()
# fr.close()

In [ ]:
if args.mode == 'train':
    train(model, args.datapath, args.checkpoint_path, args.train_iter, args.trajectory_length, args)
elif args.mode == 'test':
    test(model, args.datapath, args.trajectory_length, args.validation_steps, preprocess)
else:
    raise RuntimeError('undefined mode {}'.format(args.mode))

[345, 439, 346, 1432, 929, 434, 399, 520, 308, 1447] 6599 2980
current trajectory is:  ['office/Hard/P003/', 'seasonsforest/Hard/P006/', 'neighborhood/Hard/P009/', 'seasonsforest/Easy/P002/', 'hospital/Easy/P012/', 'westerndesert/Hard/P007/', 'carwelding/Easy/P006/', 'office2/Easy/P010/', 'abandonedfactory_night/Easy/P012/', 'ocean/Easy/P010/', 'westerndesert/Hard/P001/', 'hospital/Easy/P014/', 'hospital/Easy/P035/', 'hospital/Hard/P038/', 'office2/Easy/P007/', 'hospital/Hard/P049/']
[300, 300, 301, 301, 302, 303, 304, 304, 305, 306, 307, 307, 307, 308, 310, 311] 4876 4672


50it [13:57, 15.00s/it]
0it [00:00, ?it/s]

trying again
trying again


0it [01:39, ?it/s]

breaks at batch_idx:  0


...

Observe if the numbers match:  150 150 150


200it [54:42, 14.12s/it]
0it [01:36, ?it/s]

breaks at batch_idx:  0


Observe if the numbers match:  200 200 200


250it [1:07:48, 14.10s/it]
273it [1:14:37, 14.58s/it]

In [ ]:
!ls ./training_progess/


In [ ]:

# bc = container_client.get_blob_client(blob=image_file)
# data = None
# while data is None:
#     data = bc.download_blob(timeout=5)

# ee = io.BytesIO(data.content_as_bytes())
# img=cv2.imdecode(np.asarray(bytearray(ee.read()),dtype=np.uint8), cv2.IMREAD_COLOR)
# im_rgb = img[:, :, [2, 1, 0]] # BGR2RGB
# return im_rgb

In [ ]:
torch.get_num_interop_threads(), torch.get_num_threads()

In [ ]:

# import gc
gc.collect()

In [ ]:
!nvidia-smi

# New Section